In [1]:

import sys
sys.path.append('..')
sys.path.append('../ehrshot')
import argparse
import json
import os
from typing import Any, Dict, List, Optional, Tuple, Union
import numpy as np
import collections
import pandas as pd
import pickle
from datetime import datetime
from loguru import logger
# from utils import (
#     LABELING_FUNCTION_2_PAPER_NAME,
#     SHOT_STRATS,
#     MODEL_2_INFO,
#     get_labels_and_features, 
#     process_chexpert_labels, 
#     convert_multiclass_to_binary_labels,
#     CHEXPERT_LABELS, 
#     LR_PARAMS, 
#     XGB_PARAMS, 
#     RF_PARAMS,
#     ProtoNetCLMBRClassifier, 
#     get_patient_splits_by_idx
# )


# import femr
# import femr.datasets
# from femr.labelers import load_labeled_patients, LabeledPatients

import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
torch.manual_seed(42)
from tqdm import tqdm

In [2]:
data_dir = '../white_box_common_data/'
data_file = ['general_operation_with_index.csv', 'general_operation_with_index_v2.csv','new_diagnose_with_index.csv', 'lab_test_with_index.csv']

multi_task_dir = 'same_time_data/'

multi_task_name = ['general_operation_v1', 'general_operation_v2','new_diagnose', 'lab_test']

general_operation_with_index.csv
3617
6491

general_operation_with_index_v2.csv
3268
3268

new_diagnose_with_index.csv
1916
2794

lab_test_with_index.csv
5691
152331



In [17]:
temp_df

,patient_id,prediction_time,value_anemia,value_hypoglycemia,value_hyponatremia,value_thrombocytopenia,value_hyperkalemia,feature_index
0,115973769,2011-10-01 11:19:00,0,0,0,0,0,143855
1,115973769,2011-10-05 14:24:00,0,0,0,0,0,143858
2,115973769,2011-10-06 02:59:00,1,0,0,0,0,143860
3,115973769,2011-10-08 04:49:00,0,0,0,1,0,143861
4,115973769,2012-08-24 14:44:00,0,0,0,0,0,143863
...,...,...,...,...,...,...,...,...
152326,115967121,2023-02-10 09:23:00,1,0,0,0,0,207392
152327,115967121,2023-02-12 08:44:00,1,0,0,0,0,207393
152328,115967121,2023-02-16 10:45:00,1,0,0,0,0,207394
152329,115967121,2023-02-23 09:17:00,1,0,0,0,0,207395


In [18]:
df_split = pd.read_csv('../EHRSHOT_ASSETS/splits/person_id_map.csv')

with open('../EHRSHOT_ASSETS/features/clmbr_features.pkl', 'rb') as f:
    data = pickle.load(f)
features = data['data_matrix']

In [19]:
train_person_id = df_split.loc[df_split['split'] == 'train', 'omop_person_id'].values
test_person_id = df_split.loc[df_split['split'] == 'test', 'omop_person_id'].values
val_person_id = df_split.loc[df_split['split'] == 'val', 'omop_person_id'].values

In [20]:
def get_dataset_by_split(df, train_person_id, test_person_id, val_person_id):
    train_mask = df['patient_id'].isin(train_person_id)
    filtered_train_indices = df.index[train_mask]

    test_mask = df['patient_id'].isin(test_person_id)
    filtered_test_indices = df.index[test_mask]

    val_mask = df['patient_id'].isin(val_person_id)
    filtered_val_indices = df.index[val_mask]

    y_train = df.loc[filtered_train_indices]
    y_test = df.loc[filtered_test_indices]
    y_val = df.loc[filtered_val_indices]

    y_train_index = y_train['feature_index'].values
    y_test_index = y_test['feature_index'].values
    y_val_index = y_val['feature_index'].values

    x_train = pd.DataFrame(features[y_train_index])
    x_test = pd.DataFrame(features[y_test_index])
    x_val = pd.DataFrame(features[y_val_index])

    return x_train, x_test, x_val, y_train, y_test, y_val

In [26]:
for i in range(len(data_file)):

    df_name = data_file[i]
    print(df_name)
    df = pd.read_csv(os.path.join(data_dir, df_name))

    all_task_dir = os.path.join(multi_task_dir, multi_task_name[i])
    if not os.path.exists(all_task_dir):
        os.makedirs(all_task_dir)
        print(all_task_dir)
    x_train, x_test, x_val, y_train, y_test, y_val = get_dataset_by_split(df, train_person_id, test_person_id, val_person_id)
#     x_train.to_csv(os.path.join(all_task_dir, 'x_train.csv'), index=False)
#     x_test.to_csv(os.path.join(all_task_dir, 'x_test.csv'), index=False)
#     x_val.to_csv(os.path.join(all_task_dir, 'x_val.csv'), index=False)
#     y_train.to_csv(os.path.join(all_task_dir, 'y_train.csv'), index=False)
#     y_test.to_csv(os.path.join(all_task_dir, 'y_test.csv'), index=False)
#     y_val.to_csv(os.path.join(all_task_dir, 'y_val.csv'), index=False)
    print(f'{y_train.shape[0]} / {y_val.shape[0]} / {y_test.shape[0]} ')

general_operation_with_index.csv
2402 / 2052 / 2037 
general_operation_with_index_v2.csv
1192 / 1037 / 1039 
new_diagnose_with_index.csv
959 / 956 / 879 
lab_test_with_index.csv
59983 / 44928 / 47420 


In [23]:
for i in range(len(data_file)):
    f = data_file[i]
    temp_df = pd.read_csv(os.path.join(data_dir, f))
    print(f)
    print(len(temp_df['patient_id'].unique()))
    print(len(temp_df))
    print('')

general_operation_with_index.csv
3617
6491

general_operation_with_index_v2.csv
3268
3268

new_diagnose_with_index.csv
1916
2794

lab_test_with_index.csv
5691
152331

